In [1]:
import geopandas as gpd
import networkx as nx
import osmnx as ox
import pandas as pd

In [2]:
stations = gpd.read_file('station in road new.shp')
edges = gpd.read_file('edges.shp')

In [3]:
stations

,commonName,lon,lat,Start.stat,End.statio,Counts,End.stat_1,End.stat_2,osm_id,code,...,layer,bridge,tunnel,n,distance,feature_x,feature_y,nearest_x,nearest_y,geometry
0,"River Street , Clerkenwell",-0.109970,51.529163,"River Street , Clerkenwell","Driffield Road, Old Ford",15,-0.037366,51.534137,4253797,5122,...,0,F,F,1,5.825538,531201.770244,182831.750641,531204.957572,182826.874389,POINT (531204.958 182826.874)
1,"River Street , Clerkenwell",-0.109970,51.529163,"River Street , Clerkenwell","Dock Street, Wapping",12,-0.068161,51.509786,4253797,5122,...,0,F,F,1,5.825538,531201.770244,182831.750641,531204.957572,182826.874389,POINT (531204.958 182826.874)
2,"River Street , Clerkenwell",-0.109970,51.529163,"River Street , Clerkenwell","Red Lion Street, Holborn",10,-0.116550,51.518240,4253797,5122,...,0,F,F,1,5.825538,531201.770244,182831.750641,531204.957572,182826.874389,POINT (531204.958 182826.874)
3,"River Street , Clerkenwell",-0.109970,51.529163,"River Street , Clerkenwell","Museum of London, Barbican",9,-0.096496,51.517821,4253797,5122,...,0,F,F,1,5.825538,531201.770244,182831.750641,531204.957572,182826.874389,POINT (531204.958 182826.874)
4,"River Street , Clerkenwell",-0.109970,51.529163,"River Street , Clerkenwell","Fore Street Avenue, Guildhall",8,-0.091401,51.518093,4253797,5122,...,0,F,F,1,5.825538,531201.770244,182831.750641,531204.957572,182826.874389,POINT (531204.958 182826.874)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32057,"Coomer Place, West Kensington",-0.202038,51.483570,"Coomer Place, West Kensington","Golden Square, Soho",1,-0.137043,51.511897,98023642,5141,...,0,F,F,1,3.959992,524940.411327,177600.794610,524943.848010,177602.762032,POINT (524943.848 177602.762)
32058,"Coomer Place, West Kensington",-0.202038,51.483570,"Coomer Place, West Kensington","Imperial Road, Sands End",1,-0.187278,51.475142,98023642,5141,...,0,F,F,1,3.959992,524940.411327,177600.794610,524943.848010,177602.762032,POINT (524943.848 177602.762)
32059,"Coomer Place, West Kensington",-0.202038,51.483570,"Coomer Place, West Kensington","Millennium Hotel, Mayfair",1,-0.151126,51.510919,98023642,5141,...,0,F,F,1,3.959992,524940.411327,177600.794610,524943.848010,177602.762032,POINT (524943.848 177602.762)
32060,"Coomer Place, West Kensington",-0.202038,51.483570,"Coomer Place, West Kensington","Pembridge Villas, Notting Hill",1,-0.197524,51.511084,98023642,5141,...,0,F,F,1,3.959992,524940.411327,177600.794610,524943.848010,177602.762032,POINT (524943.848 177602.762)


In [4]:
stations = stations.to_crs("EPSG:27700")
edges = edges.to_crs("EPSG:27700")

In [5]:
colnmns =['commonName','nearest_x','nearest_y','geometry']
stations = stations[colnmns]

In [6]:
stations = stations.drop_duplicates(subset=['commonName'])
stations= stations.reset_index(drop=True)
stations['station_id'] = stations.reset_index().index + 1
stations

,commonName,nearest_x,nearest_y,geometry,station_id
0,"River Street , Clerkenwell",531204.957572,182826.874389,POINT (531204.958 182826.874),1
1,"Phillimore Gardens, Kensington",525203.530183,179389.872735,POINT (525203.530 179389.873),2
2,"Christopher Street, Liverpool Street",532984.115424,182001.761003,POINT (532984.115 182001.761),3
3,"St. Chad's Street, King's Cross",530435.413114,182912.683769,POINT (530435.413 182912.684),4
4,"Sedding Street, Sloane Square",528051.439096,178742.194456,POINT (528051.439 178742.194),5
...,...,...,...,...,...
787,"South Bermondsey Station, Bermondsey",534950.877189,178362.716371,POINT (534950.877 178362.716),788
788,"Crimscott Street, Bermondsey",533454.753045,179155.642693,POINT (533454.753 179155.643),789
789,"Brandon Street, Walworth",532599.977561,178412.274200,POINT (532599.978 178412.274),790
790,"The Blue, Bermondsey",534602.313462,178815.433593,POINT (534602.313 178815.434),791


In [7]:
stations.set_index('commonName', inplace=True)

In [8]:
stations['x'] = stations.geometry.x
stations['y'] = stations.geometry.y

In [9]:
#load the flow according the travel data
df = pd.read_csv('output_new.csv')
df

,Start.station,End.station,Counts
0,"Ackroyd Drive, Bow","Maplin Street, Mile End",98
1,"Waterloo Station 3, Waterloo","Newgate Street , St. Paul's",85
2,"Rainville Road, Hammersmith","Barons Court Station, West Kensington",77
3,"Waterloo Station 3, Waterloo","New Fetter Lane, Holborn",70
4,"Waterloo Station 3, Waterloo","Cheapside, Bank",68
...,...,...,...
32300,"Nantes Close, Wandsworth","St. John's Road, Clapham Junction",1
32301,"Nantes Close, Wandsworth","St Mary's Hospital, Paddington",1
32302,"Nantes Close, Wandsworth","South Park, Sands End",1
32303,"Crimscott Street, Bermondsey","Braham Street, Aldgate",1


In [10]:
df_nightpeak =  pd.read_csv('output_nightpeak.csv')

In [11]:
import networkx as nx

G = nx.DiGraph() 

for index, row in df.iterrows():
    origin = row['Start.station']
    destination = row['End.station']
    flow = row['Counts']
    
    # 为边添加权重，这里使用流量的倒数作为权重。
    G.add_edge(origin, destination, weight=flow )

In [12]:
import networkx as nx

G1 = nx.DiGraph() 

for index, row in df_nightpeak.iterrows():
    origin = row['Start.station']
    destination = row['End.station']
    flow = row['Counts']
    
    
    G1.add_edge(origin, destination, weight=flow )

In [13]:
# morning peak
centrality_eig = nx.eigenvector_centrality(G)

centrality_deg = nx.degree_centrality(G)

In [14]:
# evening peak
centrality_deg_night = nx.degree_centrality(G1)

centrality_eig_night = nx.eigenvector_centrality(G1)

In [15]:

centrality_eig_series = pd.Series(centrality_eig)
centrality_deg_series = pd.Series(centrality_deg)




stations = pd.concat([stations, centrality_eig_series.rename('centrality_eig'), centrality_deg_series.rename('centrality_deg')], axis=1)


In [16]:



centrality_eig_series_night = pd.Series(centrality_eig_night)
centrality_deg_series_night = pd.Series(centrality_deg_night)


stations_night = pd.concat([stations, centrality_eig_series_night.rename('centrality_eig_night'), centrality_deg_series_night.rename('centrality_deg_night')], axis=1)


stations_sorted_night = stations_night.sort_values(by='centrality_deg_night', ascending=False)



stations_night = stations_night.dropna()
stations_night

,nearest_x,nearest_y,geometry,station_id,x,y,centrality_eig,centrality_deg,centrality_eig_night,centrality_deg_night
"River Street , Clerkenwell",531204.957572,182826.874389,POINT (531204.958 182826.874),1.0,531204.957572,182826.874389,0.009375,0.056040,0.020411,0.125778
"Phillimore Gardens, Kensington",525203.530183,179389.872735,POINT (525203.530 179389.873),2.0,525203.530183,179389.872735,0.025724,0.112080,0.038805,0.287671
"Christopher Street, Liverpool Street",532984.115424,182001.761003,POINT (532984.115 182001.761),3.0,532984.115424,182001.761003,0.064703,0.130760,0.022003,0.193026
"St. Chad's Street, King's Cross",530435.413114,182912.683769,POINT (530435.413 182912.684),4.0,530435.413114,182912.683769,0.019062,0.056040,0.033878,0.164384
"Sedding Street, Sloane Square",528051.439096,178742.194456,POINT (528051.439 178742.194),5.0,528051.439096,178742.194456,0.050677,0.181818,0.048488,0.367372
...,...,...,...,...,...,...,...,...,...,...
"South Bermondsey Station, Bermondsey",534950.877189,178362.716371,POINT (534950.877 178362.716),788.0,534950.877189,178362.716371,0.006973,0.048568,0.019173,0.089664
"Crimscott Street, Bermondsey",533454.753045,179155.642693,POINT (533454.753 179155.643),789.0,533454.753045,179155.642693,0.012939,0.090909,0.024701,0.130760
"Brandon Street, Walworth",532599.977561,178412.274200,POINT (532599.978 178412.274),790.0,532599.977561,178412.274200,0.004290,0.048568,0.031255,0.141968
"The Blue, Bermondsey",534602.313462,178815.433593,POINT (534602.313 178815.434),791.0,534602.313462,178815.433593,0.009217,0.039851,0.020400,0.078456


In [17]:
stations_sorted = stations.sort_values(by='centrality_deg', ascending=False)
stations = stations.dropna()

In [18]:
#shp --centrality in the morning peak
#stations_gdf = gpd.GeoDataFrame(stations, geometry='geometry')
#stations_gdf.to_file("stations_new_weight.shp")

/tmp/ipykernel_134/3300781405.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  stations_gdf.to_file("stations_new_weight.shp")


In [19]:
#shp --centrality in the evening peak
stations_gdf_night = gpd.GeoDataFrame(stations_night, geometry='geometry')
stations_gdf_night.to_file("stations_new_weight_night.shp")

/tmp/ipykernel_134/2089722489.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  stations_gdf_night.to_file("stations_new_weight_night.shp")
